In [78]:
import pandas as pd
import geopandas as gpd
import sqlite3
import spatialite
from shapely import wkt

In [88]:
df = gpd.read_file(r'geojson\brasil.json')
df = df.drop(['description'], axis=1)
df['geometry'] = df['geometry'].apply(wkt.dumps)
df

D:\miniconda3\envs\bd2\lib\site-packages\geopandas\geodataframe.py:1350: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


,id,name,geometry
0,1100015,Alta Floresta D'Oeste,POLYGON ((-62.1820888569999966 -11.86685978779...
1,1100023,Ariquemes,POLYGON ((-62.5359497334000025 -9.731823527199...
2,1100031,Cabixi,POLYGON ((-60.3993982596999999 -13.45584182760...
3,1100049,Cacoal,POLYGON ((-61.0005091458999971 -11.39796015469...
4,1100056,Cerejeiras,POLYGON ((-61.5004673574999998 -13.00392326040...
...,...,...,...
5559,5222005,Vianópolis,POLYGON ((-48.4312481047999981 -16.62754663049...
5560,5222054,Vicentinópolis,POLYGON ((-49.8500478799999982 -17.57682406609...
5561,5222203,Vila Boa,POLYGON ((-47.1101851052000029 -14.67149996540...
5562,5222302,Vila Propício,POLYGON ((-48.7512448486000025 -14.90460640360...


In [68]:
# Criando/conectando database
conn = sqlite3.connect('citiesXD.db')
c = conn.cursor()
c.execute('SELECT load_extension("mod_spatialite.dll")')
c.execute('SELECT InitSpatialMetaData()')

In [90]:
c.execute('create table cities (id integer primary key, name text)')
c.execute("SELECT AddGeometryColumn('cities', 'geometry', 4326, 'POLYGON', 'XY');")
c.execute("SELECT CreateSpatialIndex('cities', 'geometry');")
conn.commit()

In [91]:
c.execute('pragma table_info(cities)')
c.fetchall()

[(0, 'id', 'integer', 0, None, 1),
 (1, 'name', 'text', 0, None, 0),
 (2, 'geometry', 'POLYGON', 0, None, 0)]

In [86]:
c.execute("SELECT * FROM geometry_columns WHERE f_table_name LIKE 'cities'")
c.fetchall()

[('cities', 'geometry', 3, 2, 4326, 1)]

In [92]:
df.to_sql('cities', conn, if_exists='replace',index=False)

In [89]:
c.execute('DROP TABLE cities')
conn.commit()

In [93]:
#Fechando conexão
c.close()
conn.close()